In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, LongType, TimestampType
from dotenv import load_dotenv
import os
import json
import time
import boto3
from pyspark.sql import SparkSession
from datetime import datetime 

In [2]:
# Load environment variables from .env file
load_dotenv()

# Initialize AWS credentials from the .env file
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_SESSION_TOKEN = os.getenv('AWS_SESSION_TOKEN')
AWS_REGION = os.getenv('AWS_REGION')

In [9]:
spark.stop()

In [4]:
from pyspark.sql import SparkSession

# Path to your local JAR files
local_jars = "/Users/borja/Documents/Somniumrema/projects/de/route_optimizer/jars/aws-java-sdk-kinesis-1.12.364.jar"

# Initialize Spark session with Delta and S3 settings
spark = SparkSession.builder \
    .appName("KinesisToDeltaLake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0,org.apache.hadoop:hadoop-aws:3.3.2,com.amazonaws:aws-java-sdk-bundle:1.11.1026") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY_ID) \
    .config("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.session.token", AWS_SESSION_TOKEN) \
    .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
    .config("spark.sql.files.maxPartitionBytes", "134217728") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

# Optional: Adjust logging level
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/Users/borja/Library/Caches/pypoetry/virtualenvs/route-optimizer-AqO2e-Ud-py3.11/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/borja/.ivy2/cache
The jars for the packages stored in: /Users/borja/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-48b4f624-75bb-4103-a3e3-ff9351b571f4;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 153ms :: artifacts dl 7ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-ru

In [5]:
# Initialize boto3 client for Kinesis with your credentials
kinesis_client = boto3.client(
    'kinesis',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    region_name=AWS_REGION
)

In [6]:
# Define the schema
schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("total_weight", DoubleType(), True),
    StructField("total_volume", DoubleType(), True),
    StructField("total_price", DoubleType(), True),
    StructField("order_timestamp", TimestampType(), True),
    StructField("status", StringType(), True),
    StructField("lat", DoubleType(), True),
    StructField("lon", DoubleType(), True)
])

In [7]:
# Function to convert the timestamps to proper datetime objects
def convert_timestamps(orders):
    for order in orders:
        if isinstance(order['order_timestamp'], str):
            # Only convert if the timestamp is a string
            order['order_timestamp'] = datetime.strptime(order['order_timestamp'], '%Y-%m-%d %H:%M:%S')
    return orders

# Function to convert the location field from MapType to StructType
def transform_location(order):
    if isinstance(order['location'], dict):
        location_data = order['location']
        order['location'] = {
            "address": location_data['address'],
            "lat": float(location_data['lat']),
            "lon": float(location_data['lon'])
        }
    return order

# Function to get the shard iterator
def get_shard_iterator(stream_name, shard_id):
    response = kinesis_client.get_shard_iterator(
        StreamName=stream_name,
        ShardId=shard_id,
        ShardIteratorType='LATEST'  # or 'LATEST' for new records
    )
    return response['ShardIterator']

# Function to read records from the Kinesis stream
def read_kinesis_records(stream_name, shard_iterator):
    while True:
        response = kinesis_client.get_records(ShardIterator=shard_iterator, Limit=100)
        records = response['Records']
        for record in records:
            # No need to base64 decode, just parse the data directly
            order_data = record['Data']
            order = json.loads(order_data)
            print("Received order:", order)

        # Update the shard iterator for the next batch of records
        shard_iterator = response['NextShardIterator']

        # Sleep to avoid hitting API rate limits
        time.sleep(1)

In [8]:
import time
import json

# Initialize buffer, limits, and thresholds
order_buffer = []
weight_threshold = 700  # Example weight threshold (can be adjusted)
volume_threshold = 500  # Example volume threshold (can be adjusted)
time_threshold = 60 * 45 # Example time threshold (e.g., 30 minutes)
buffer_limit = 10000      # Optional, max number of orders in the buffer
last_flush_time = time.time()

# S3 path for Delta table
delta_table_path = "s3a://orders-for-dispatch/dispatching"

# Dispatcher function to read Kinesis records, accumulate, and process them
def dispatcher(shard_iterator):
    global order_buffer, last_flush_time

    while True:
        # Fetch records from Kinesis using shard_iterator
        response = kinesis_client.get_records(ShardIterator=shard_iterator)
        records = response['Records']

        # Parse and accumulate orders
        for record in records:
            order_data = json.loads(record['Data'])
            order_buffer.append(order_data)
            print(f"Accumulated order: {order_data}")

        # Convert timestamps if necessary
        order_buffer = convert_timestamps(order_buffer)

        # Calculate accumulated weight and volume
        total_weight = sum(order['total_weight'] for order in order_buffer)
        total_volume = sum(order['total_volume'] for order in order_buffer)
        time_elapsed = time.time() - last_flush_time

        # Check if any thresholds are met (weight, volume, or time)
        if total_weight >= weight_threshold or total_volume >= volume_threshold or time_elapsed >= time_threshold:
            print(f"Threshold met: Dispatching {len(order_buffer)} orders.")
            
            # Update the status of all orders to 'DISPATCH_READY'
            for order in order_buffer:
                order['status'] = 'READY_FOR_DISPATCH'

            # Convert buffer to DataFrame and write to Delta table
            df = spark.createDataFrame(order_buffer, schema=schema)
            df.write.format("delta").mode("append").save(delta_table_path)
            print(f"Saved {len(order_buffer)} orders to Delta table.")
            
            # Clear the buffer and reset flush time
            order_buffer.clear()
            last_flush_time = time.time()

        # Update shard iterator for the next batch
        shard_iterator = response['NextShardIterator']
        time.sleep(2)  # Sleep to avoid rate limits

# Get the shard iterator
stream_name = 'OrderStreamForDispatching'  # Replace with your stream name
shard_id = 'shardId-000000000000'  # Get the shard ID from the stream's details
shard_iterator = get_shard_iterator(stream_name, shard_id)

# Start the dispatcher
dispatcher(shard_iterator)


Accumulated order: {'order_id': 'a582af42-53c8-42bd-b240-4c7eabdc776f', 'customer_id': 'cus-42fa4fe5-ce3a-45f9-ad18-6c1d61b179b4', 'total_weight': 27.4355171659557, 'total_volume': 381.54823546388553, 'total_price': 648.89984812418, 'order_timestamp': '2024-10-05 20:10:40', 'status': 'RECEIVED', 'lat': 40.348948379328654, 'lon': -3.687971612608781}
Accumulated order: {'order_id': '44e06181-b121-459f-9b33-862d8f36e5df', 'customer_id': 'cus-32f7d4c8-01c3-4796-aa0d-be7879b1d408', 'total_weight': 35.18021419665379, 'total_volume': 423.5040864509453, 'total_price': 874.6671088619494, 'order_timestamp': '2024-10-05 20:10:52', 'status': 'RECEIVED', 'lat': 40.61135913254186, 'lon': -3.8292090835154835}
Threshold met: Dispatching 2 orders.


24/10/05 20:10:56 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/10/05 20:11:06 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Saved 2 orders to Delta table.
Accumulated order: {'order_id': 'b8e451c7-4d4c-428f-8105-5fab9a354914', 'customer_id': 'cus-c2bcb3ae-8118-4e92-8620-2063da2da939', 'total_weight': 91.2790296088143, 'total_volume': 362.2745298883178, 'total_price': 246.82891145812798, 'order_timestamp': '2024-10-05 20:11:11', 'status': 'RECEIVED', 'lat': 40.59457455696566, 'lon': -3.765600802671365}
Accumulated order: {'order_id': 'bd5d6757-54b2-427f-b1bd-916761482b7a', 'customer_id': 'cus-b059a6a0-5703-4198-bb2b-cd2a13146de9', 'total_weight': 60.513459564260174, 'total_volume': 429.17743077954367, 'total_price': 271.4189290462663, 'order_timestamp': '2024-10-05 20:11:34', 'status': 'RECEIVED', 'lat': 40.59588071338949, 'lon': -3.8057194310823563}
Threshold met: Dispatching 2 orders.


Saved 2 orders to Delta table.
Accumulated order: {'order_id': '73a4e65e-e450-4e9f-b321-30371d43837a', 'customer_id': 'cus-516f5414-a7e1-474d-8108-f00fbfdfbaec', 'total_weight': 75.6963534018612, 'total_volume': 95.91137991366476, 'total_price': 75.30102099140262, 'order_timestamp': '2024-10-05 20:11:53', 'status': 'RECEIVED', 'lat': 40.64422320695071, 'lon': -3.8012655984148522}
Accumulated order: {'order_id': 'c3bb2282-cad9-4484-9104-d1a6c617cf6c', 'customer_id': 'cus-f176e720-c9ed-4340-a965-290c23599084', 'total_weight': 89.93411221339048, 'total_volume': 71.47627779318901, 'total_price': 894.8478004028301, 'order_timestamp': '2024-10-05 20:12:12', 'status': 'RECEIVED', 'lat': 40.481463770970855, 'lon': -3.8592446256198474}
Accumulated order: {'order_id': 'ae028421-cf5f-4eec-a153-510dd897caaf', 'customer_id': 'cus-62209290-b925-4fc9-b012-c050e7d008e3', 'total_weight': 72.00706126721471, 'total_volume': 225.39375256384164, 'total_price': 135.7980091676473, 'order_timestamp': '2024-10

Saved 4 orders to Delta table.
Accumulated order: {'order_id': 'b404cbbd-65da-4552-a062-6d0cc054de3b', 'customer_id': 'cus-e5385b0c-2083-45a1-8df1-67d259af1200', 'total_weight': 31.428246018807688, 'total_volume': 438.4141903472967, 'total_price': 970.307187012231, 'order_timestamp': '2024-10-05 20:13:18', 'status': 'RECEIVED', 'lat': 40.3553672160869, 'lon': -3.8273277684294724}
Accumulated order: {'order_id': 'b80745c0-1447-4587-9055-f7d07f153f64', 'customer_id': 'cus-9df343ac-f9e8-4c27-bd33-55d57fa7d46a', 'total_weight': 86.24744493868062, 'total_volume': 461.43552609580513, 'total_price': 667.2547260565223, 'order_timestamp': '2024-10-05 20:13:43', 'status': 'RECEIVED', 'lat': 40.47596705459622, 'lon': -3.5971934962812417}
Threshold met: Dispatching 2 orders.


...

Saved 2 orders to Delta table.
Accumulated order: {'order_id': '0bb84045-4add-4e27-83a3-f2aec5caa2df', 'customer_id': 'cus-447614fa-52ba-4fa5-9340-2678dcbe96ad', 'total_weight': 83.73579088546752, 'total_volume': 31.616799641092225, 'total_price': 760.3526086050654, 'order_timestamp': '2024-10-05 20:28:19', 'status': 'RECEIVED', 'lat': 40.44640880300795, 'lon': -3.6986996650252233}
Accumulated order: {'order_id': '4317f9a8-7abc-4364-aed2-aa9cfb9574ed', 'customer_id': 'cus-7950ef3e-8573-44fa-8e18-ac7e81a45fb0', 'total_weight': 85.3651372965385, 'total_volume': 199.91015828228706, 'total_price': 212.86381560978074, 'order_timestamp': '2024-10-05 20:28:43', 'status': 'RECEIVED', 'lat': 40.485635935744476, 'lon': -3.7310533711174263}
Accumulated order: {'order_id': '423ba676-7192-4812-b94b-b2754073e4fe', 'customer_id': 'cus-2d4f3f10-f1b0-41e5-a8b5-69f8ce58d624', 'total_weight': 75.35676140026804, 'total_volume': 118.74790285992128, 'total_price': 360.911636216236, 'order_timestamp': '2024-

Saved 4 orders to Delta table.
Accumulated order: {'order_id': '761c36b4-9947-47e7-b120-013cb2c7d873', 'customer_id': 'cus-5de58b93-4f99-4a24-8231-17f86f32b3bf', 'total_weight': 33.78508959223261, 'total_volume': 42.813537055517244, 'total_price': 194.86764969084203, 'order_timestamp': '2024-10-05 20:29:49', 'status': 'RECEIVED', 'lat': 40.43635264690259, 'lon': -3.818244706869084}
Accumulated order: {'order_id': 'ce8b56b5-3b7c-4e62-bbe9-db60b422b763', 'customer_id': 'cus-7fbc8eb2-324e-4874-93eb-364475f51b8b', 'total_weight': 51.900029193141634, 'total_volume': 63.09417476343221, 'total_price': 466.6595915050284, 'order_timestamp': '2024-10-05 20:30:01', 'status': 'RECEIVED', 'lat': 40.38107837189181, 'lon': -3.8094346553617724}
Accumulated order: {'order_id': 'e219e1af-397d-44e0-94a9-5459787456de', 'customer_id': 'cus-cc4157eb-7e09-4d5a-95ef-0ad28609d91e', 'total_weight': 58.258605105239525, 'total_volume': 45.918506740210894, 'total_price': 956.4549301624764, 'order_timestamp': '2024

Saved 4 orders to Delta table.
Accumulated order: {'order_id': '75d76964-45e0-4ea8-9024-4b81eea21ad9', 'customer_id': 'cus-dce58a2d-f3a8-425f-a74b-ff1b338dd73a', 'total_weight': 34.88869214481892, 'total_volume': 473.89455567689475, 'total_price': 917.8236686436388, 'order_timestamp': '2024-10-05 20:31:17', 'status': 'RECEIVED', 'lat': 40.62527353394663, 'lon': -3.7068902406227497}
Accumulated order: {'order_id': '81fc91ae-871d-47d2-bd94-3f4b398cfce6', 'customer_id': 'cus-eaf6010e-e9f2-46a6-bb51-06206b6a7e6f', 'total_weight': 8.564754594209866, 'total_volume': 468.94692487186876, 'total_price': 922.1419792625402, 'order_timestamp': '2024-10-05 20:31:42', 'status': 'RECEIVED', 'lat': 40.605588953511734, 'lon': -3.6059403947532984}
Threshold met: Dispatching 2 orders.


Saved 2 orders to Delta table.
Accumulated order: {'order_id': '6d959e84-d953-471a-8598-5d713243406d', 'customer_id': 'cus-8ae5bb0e-0c1e-4ee1-9c85-14661912a79e', 'total_weight': 30.563421310640226, 'total_volume': 357.1335118447459, 'total_price': 608.1940051190887, 'order_timestamp': '2024-10-05 20:32:00', 'status': 'RECEIVED', 'lat': 40.55634181056684, 'lon': -3.7031527686828083}
Accumulated order: {'order_id': '75919185-4cf1-4cf3-8a23-9ad70d3293da', 'customer_id': 'cus-84e414ab-76f0-47f7-8d26-a993b00d6660', 'total_weight': 2.004647525186456, 'total_volume': 63.22251989131597, 'total_price': 449.7262244505143, 'order_timestamp': '2024-10-05 20:32:20', 'status': 'RECEIVED', 'lat': 40.621050147582785, 'lon': -3.5864777693850223}


KeyboardInterrupt: 

In [9]:
spark.stop()